# Récupérer les URI originelles

Ce script a été utilisé pour vérifier les URI originelles des classes et propriétés des différents espaces de nom externes à OntoME depuis les ontologies officielles téléchargées dans le dossier [input](input).

> *Getting original URIs: This script was used to check original URIs for the classes and properties in each namespace external to OntoMe, from the official ontologies downloaded into the [input](input) folder.*

### Imports et associations

- Les identifiants uniques et espaces de noms de chaque classe et propriété d'OntoMe sont importés depuis un fichier Json.
- Chacun est associé au fichier ou lien contenant l'ontologie officielle en XML-RDF ou TTL.

> *Imports and matches:*
> - *The unique identifiers and namespaces of each class and property in OntoMe are imported from a Json file.*
> - *Each is matched to the file or link containing the official ontology in RDF-XML or TTL.*

In [1]:
import lxml.etree as et
import lxml.html as ht
from datetime import datetime
import re
from itertools import chain
import json
import csv
import rdflib


# Importer les classes et propriétés OntoME à compléter.
# Import OntoME classes and properties to be completed.
with open("input/ns_entity.json") as jsonfile:
    ontome_in = json.load(jsonfile)

# Initialiser le dictionnaire des espaces de noms + ontologies.
# Initiate the namespace + ontology dictionary.
ns_with_ontology = {}
for item in ontome_in[0].keys():
    if "ontome" not in item:
        print(item)
        ns_with_ontology[item] = ""

# Association des espaces de noms à leur ontologie.
# Matching namespaces with their ontology.
ns_with_ontology["http://www.cidoc-crm.org/cidoc-crm/CRMarchaeo/"] = "input/CRMarchaeo_v1.4.1.rdfs"
ns_with_ontology["http://www.ics.forth.gr/isl/CRMgeo/"] = "input/CRMgeo_v1_2.rdfs"
ns_with_ontology["http://www.cidoc-crm.org/cidoc-crm/"] = "input/CIDOC_CRM_v7.1.1.rdfs.rdf"
ns_with_ontology["http://iflastandards.info/ns/fr/frbr/frbroo/"] = "input/FRBR2.4-draft.rdfs"
ns_with_ontology["http://data.doremus.org/ontology#"] = "input/doremus.ttl"
ns_with_ontology["http://www.opengis.net/ont/geosparql#"] = "http://www.opengis.net/ont/geosparql#"

# ignoring MoMaf
ns_with_ontology["http://momaf-data.utu.fi/"] = ""

# sci: none -> S2_Sample_Taking
ns_with_ontology["http://www.cidoc-crm.org/cidoc-crm/CRMsci/"] = ""

http://www.cidoc-crm.org/cidoc-crm/CRMarchaeo/
http://www.ics.forth.gr/isl/CRMgeo/
http://momaf-data.utu.fi/
http://www.opengis.net/ont/geosparql#
http://www.cidoc-crm.org/cidoc-crm/CRMsci/
http://www.cidoc-crm.org/cidoc-crm/
http://iflastandards.info/ns/fr/frbr/frbroo/
http://data.doremus.org/ontology#


### FONCTION : Découper les URI...
...pour obtenir le nom de la classe/entité, son identifiant dans l'espace de nom, et l'espace de nom, séparés. Adapté du script [graph_to_ontome](graph_to_ontome.ipynb)

> *FUNCTION: cut URIs to obtain the name of the class/entity, its identifier in its own namespace and the namespace itself, apart from each other. Adapted from the script [graph_to_ontome](graph_to_ontome.ipynb).*

EX :

| Variable | Value |
| -------: | :----- |
| about | rdflib.term.URIRef('http://iflastandards.info/ns/fr/frbr/frbroo/F24_Publication_Expression') |
| about_val | 'http://iflastandards.info/ns/fr/frbr/frbroo/F24_Publication_Expression' |
| ns | 'http://iflastandards.info/ns/fr/frbr/frbroo/' |
| id full | 'F24_Publication_Expression' |
| id short | 'F24' |
| name | 'F24 Publication Expression' |

In [2]:
def cut_about(about, ns):
    
    # Récupérer l'URI de l'objet rdflib.Graph
    # Get the URI from the rdflib.Graph object.
    about_val = str(about)
    
    try:
        
        # Couper sur le # ou / -> espace de nom vs identifiant.
        # Cut on # or / sign -> namespace vs identifier
        if "#" in about_val:
            origns = about_val.split("#")[0] + "#"
            cp = about_val.split("#")[1]
        
        else:
            full = about_val.split("/")
            cp = full[-1]
            del full[-1]
            origns = "/".join(full)
        
        # Couper l'identifiant sur les _
        # Cut identifier on the _
        cpcut = cp.split("_")
        
        # La première partie de l'identifiant est l'identifiant court.
        # First part of identifier is short identifier.
        identifier = cpcut[0]
        
        # On l'enlève et on reconstruit la str pour récupérer le label au cas où.
        # Removing it and reconstructing the str to get the label in case.
        del cpcut[0]
        name = "_".join(cpcut)
        
        returndict = {
            'full': about_val,
            'ns':origns,
            'id full':cp,
            'id short':identifier,
            'name':name
        }
        # print(returndict)
        # print(ns, cp, identifier, name)
        return returndict
    
    
    except Exception as e:
        print(f"Exception : {about_val} -> {e}")

### Parser les ontologies

Pour chaque espace de nom associé à un fichier ou lien :
- Import de l'ontologie dans un graphe.
- Récupération des classes et propriétés de l'ontologie dans deux listes distinctes.

> *Parsing ontologies: For each namespace matched with a file or link:*
> - *The ontology is imported into a graph.*
> - *Classes and properties are filed into two separate lists.*

In [3]:
# Préparation des dictionnaires d'inventaire.
# Preparing inventory dictionaries.
all_classes = {}
all_props = {}

# Boucle sur les espaces de nom avec ontologie associée.
# Loop on namespaces with matching ontology.
for ns in ns_with_ontology.keys():
    if ns_with_ontology[ns] != "":
        
        try:
            
            # Essayer de parser l'ontologie comme du XML.
            # Try to parse ontology as XML.
            g = rdflib.Graph()
            g.parse(ns_with_ontology[ns], format="xml")
            print(f"Success (XML) -> {ns}.")
            
        except:
            try:
                
                # Si ce n'est pas du XML, essayer de parser comme du Turtle.
                # If it's not XML, try parsing as Turtle.
                g = rdflib.Graph()
                g.parse(ns_with_ontology[ns])
                print(f"Success (Turtle) -> {ns}.")
                
            except Exception as e:
                print('Error: ' + str(e))
        
        # Préparer les listes des classes et propriétés pour cet espace de nom.
        # Prepare class and property lists for this namespace.        
        classes = []
        props = []
        
        # Récupérer les URIs des objets de rdf:type owl:Class.
        # Get URIs of rdf:type owl:Class objects. 
        for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))):
            classes.append(cut_about(s, ns))
        
        # Récupérer les URI des objets de rdf:type rdfs:Class.
        # Get URIs of rdf:type rdfs:Class objects.
        for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))):
            classes.append(cut_about(s, ns))
            
        # Récupérer les URI des objets de rdf:type owl:Property.
        # Get URIs of rdf:type owl:Property objects.
        for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'))):
            props.append(cut_about(s, ns))
        
        # Récupérer les URI des objets de rdf:type owl:DatatypeProperty.
        # Get URIs of rdf:type owl:DatatypeProperty objects.
        for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#DatatypeProperty'))):
            props.append(cut_about(s, ns))
        
        # Récupérer les URI des objets de rdf:type rdf:Property.
        # Get URIs of rdf:type rdf:Property objects.
        for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'))):
            props.append(cut_about(s, ns))
        
        # Ajouter les classes et propriétés de l'espace de nom aux inventaires généraux.
        # Add the namespace classes and properties to the general intentories.
        all_classes[ns] = classes
        all_props[ns] = props

Success (XML) -> http://www.cidoc-crm.org/cidoc-crm/CRMarchaeo/.
Success (XML) -> http://www.ics.forth.gr/isl/CRMgeo/.
Success (XML) -> http://www.opengis.net/ont/geosparql#.
Success (XML) -> http://www.cidoc-crm.org/cidoc-crm/.
Success (XML) -> http://iflastandards.info/ns/fr/frbr/frbroo/.
Success (Turtle) -> http://data.doremus.org/ontology#.


### Association OntoMe - ontologies


In [4]:
# Préparation du Json de sortie.
# Preparing the output Json.
first_pass = []

# Préparation des inventaires des correspondances établies, par type d'objet.
# Preparing the inventories of established matches, by object type. 
treated_props = []
treated_classes = []

# Préparation d'un dépôt pour les correspondances manquantes.
# Preparing repository for missing matches.
missing = {}

count_treated = 0
count_missing = 0

for ns in ns_with_ontology.keys():
    if ns_with_ontology[ns] != "":
        
        # Copie des informations de l'ontologie pour limiter les allers-retours.
        # Copying the ontology information to limit trafic.
        classes = all_classes[ns]
        props = all_props[ns]
        
        # Copie des informations d'OntoME, idem.
        # Copy of OntoME informations, same.
        orig = ontome_in[0][ns]
        
        missing_classes = []
        missing_props = []
        
        result = {}
        
        # Pour chaque objet, on se base sur le dictionnaire OntoME qu'on complète
        # si la correspondance est trouvée.
        # Each object gets its OntoME dictionary as a base, and is completed
        # if a match is found.
        
        for pc_object in orig:
                
            recorded_id = pc_object['identifierInNamespace']
            ontome_id = pc_object['pkOntome']
            
            if pc_object['objectType'] == 'class':
                for classdict in classes:
                    if recorded_id in classdict.values():
                        local_pc = pc_object
                        local_pc['originalURI'] = classdict['id full']
                        first_pass.append(local_pc)
                        treated_classes.append(ontome_id)
                if ontome_id not in treated_classes:
                    missing_classes.append(pc_object)
                    count_missing += 1
                else:
                    count_treated += 1
                    """print(f"{ns} - {recorded_id} - not treated")
                else:
                    print(f"All good for - {ns} - {recorded_id}")"""
                    
            if pc_object['objectType'] == 'property':
                for propdict in props:
                    if recorded_id in propdict.values():
                        local_pc = pc_object
                        local_pc['originalURI'] = propdict['id full']
                        first_pass.append(local_pc)
                        treated_props.append(ontome_id)
                if ontome_id not in treated_props:
                    missing_props.append(pc_object)
                    count_missing += 1
                else:
                    count_treated += 1
                    """print(f"{ns} - {recorded_id} - not treated")
                else:
                    print(f"All good for - {ns} - {recorded_id}")"""
        
        # Le CIDOC-CRM est la seule ontologie avec plusieurs versions.
        # On cherche les correspondances dans la version la plus récente (7.1.1),
        # mais ensuite, pour les objets sans correspondance, on teste les
        # deux autres, d'abord l'avant-dernière (6.2), puis la première (5.0.4).
        
        # The CIDOC-CRM is the only ontology with several versions.
        # Matches are first sought within the most recent one (7.1.1),
        # then the still matchless objects are sought within the last-
        # but-one (6.2), then the first (5.0.4).
        
        if ns == "http://www.cidoc-crm.org/cidoc-crm/":
            
            # Import de la version 6.2.
            # Importing version 6.2.
            six_deux = rdflib.Graph()
            six_deux.parse('input/cidoc_crm_v6.2-2018April.rdfs', format="xml")
            
            # Import de la version 5.0.4.
            # Importing version 5.0.4.
            cinq_zero_quatre = rdflib.Graph()
            cinq_zero_quatre.parse('input/cidoc_crm_v5.0.4_official_release.rdfs', format="xml")
            
            # Objets rdfs:Class dans la version 6.2.
            # rdfs:Class objects in version 6.2.
            for missing_class in missing_classes:
                for s, p, o in six_deux.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))):
                    if missing_class['identifierInNamespace'] in cut_about(s, ns).values():
                        
                        local_pc = missing_class
                        classdict = cut_about(s, ns)
                        local_pc['originalURI'] = classdict['id full']
                        treated_classes.append(missing_class['pkOntome'])
                        missing_classes.remove(missing_class)
                        first_pass.append(local_pc)
                        count_treated += 1
                        count_missing -= 1
            
            # Objets rdf:Property dans la version 6.2.
            # rdf:Property in version 6.2.
            for missing_prop in missing_props:
                for s, p, o in six_deux.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'))):
                    if missing_prop['identifierInNamespace'] in cut_about(s, ns).values():
                        local_pc = missing_prop
                        classdict = cut_about(s, ns)
                        local_pc['originalURI'] = classdict['id full']
                        treated_props.append(missing_prop['pkOntome'])
                        missing_props.remove(missing_prop)
                        first_pass.append(local_pc)
                        count_treated += 1
                        count_missing -= 1
                        
            
            # Objets rdfs:Class dans la version 5.0.4.
            # rdfs:Class objects in version 5.0.4.
            for missing_class in missing_classes:
                for s, p, o in cinq_zero_quatre.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))):
                    if missing_class['identifierInNamespace'] in cut_about(s, ns).values():
                        
                        local_pc = missing_class
                        classdict = cut_about(s, ns)
                        local_pc['originalURI'] = classdict['id full']
                        treated_classes.append(missing_class['pkOntome'])
                        missing_classes.remove(missing_class)
                        first_pass.append(local_pc)
                        count_treated += 1
                        count_missing -= 1
            
            # Objets rdf:Property dans la version 5.0.4.
            # rdf:Property in version 5.0.4.
            for missing_prop in missing_props:
                for s, p, o in cinq_zero_quatre.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'))):
                    if missing_prop['identifierInNamespace'] in cut_about(s, ns).values():
                        
                        local_pc = missing_prop
                        classdict = cut_about(s, ns)
                        local_pc['originalURI'] = classdict['id full']
                        treated_props.append(missing_prop['pkOntome'])
                        missing_props.remove(missing_prop)
                        first_pass.append(local_pc)
                        count_treated += 1
                        count_missing -= 1
        
        missing[ns] = missing_classes + missing_props

# Communication des résultats.
# Communicating results.
        
print(f"{count_treated} objets ont été correctement associés à leur URI. {count_missing} n'étaient pas dans l'ontologie.")

for namespace in missing.keys():
    print("\n")
    print(namespace)
    if missing[namespace] == []:
        print("Aucun manquant.")
    else:
        print("Manquent :")
        for item in missing[namespace]:
            print(f"\t{item['identifierInNamespace']}")

697 objets ont été correctement associés à leur URI. 15 n'étaient pas dans l'ontologie.


http://www.cidoc-crm.org/cidoc-crm/CRMarchaeo/
Aucun manquant.


http://www.ics.forth.gr/isl/CRMgeo/
Aucun manquant.


http://www.opengis.net/ont/geosparql#
Manquent :
	gmlLiteral
	wktLiteral
	hasDefaultGeometry


http://www.cidoc-crm.org/cidoc-crm/
Manquent :
	E61
	E95
	E94
	E59
	E49
	E62
	E60
	P170
	P169
	P120
	P83
	P78


http://iflastandards.info/ns/fr/frbr/frbroo/
Aucun manquant.


http://data.doremus.org/ontology#
Aucun manquant.


### Écriture des résultats dans des fichiers

Trois fichiers sont produits dans le dossier [output](output), portant le timestamp:
> *Write results into files: Three files are produced into the [output](output) folder, bearing the timestamp.*

| French | File name | *English* |
| --------: | :--------: | :------- |
| Le fichier prévu pour l'import. | `original_uris_timestamp.json` | *The file to be imported.* |
| Le même fichier en CSV pour vérification manuelle des résultats. | `original_uris_timestamp.csv` | *The same file as CSV to be able to manually check on the results.* |
| Un fichier avec l'état OntoME originel de tous les objets sans correspondance, associés à leur espace de nom. | `missing_uris_timestamp.json` | *A file with the OntoME original state of all matchless objects, along with their namespace.* |

Note : 6 objets non trouvés par le script ont été retrouvés manuellement et passés du fichier `missing_uris_timestamp.json` au fichier `original_uris_timestamp.json` avec leur correspondance (25/11/2022).
> *Note: 6 objects unfound in the script were manually found and taken from the file `missing_uris_timestamp.json` to `original_uris_timestamp.json` with their matches (11/25/2022).*

In [5]:
dt = datetime.now()
tmsp = dt.strftime("%Y%m%d_%H%M%S")

with open(f'output/original_uris_{tmsp}.json', 'w') as json_output:
    json.dump(first_pass, json_output)

# CSV pour vérifier visuellement si besoin.
with open(f'output/original_uris_{tmsp}.csv', 'w') as csv_output:
    writer = csv.DictWriter(csv_output, fieldnames = first_pass[0].keys())
    writer.writeheader()
    writer.writerows(first_pass)
    
with open(f'output/missing_uris_{tmsp}.json', 'w') as missing_output:
    json.dump(missing, missing_output)